In [1]:
from pyspark.sql.functions import lit

# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df1 = df1.withColumn('category', lit("video_games"))

df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df2 = df2.withColumn('category', lit("home_and_kitchen"))

df3 = spark.sql("select * from default.books_5_small")
df3 = df3.withColumn('category', lit("books"))

df = df1.union(df2).union(df3)

df = df.cache()

In [2]:
display(df)

reviewID,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category
0,5.0,0,true,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, but when you do it's great.",but when you do it's great.,1445040000,0,video_games
1,4.0,0,false,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,"I played it a while but it was alright. The steam was a bit of trouble. The more they move these game to steam the more of a hard time I have activating and playing a game. But in spite of that it was fun, I liked it. Now I am looking forward to anno 2205 I really want to play my way to the moon.","But in spite of that it was fun, I liked it",1437955200,0,video_games
2,3.0,0,true,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,0,video_games
3,2.0,0,true,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what I expected after having played 1602, 1503, and 1701",Two Stars,1424390400,0,video_games
4,5.0,0,true,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since its arrived",love this game,1419465600,0,video_games
5,4.0,0,true,"11 13, 2014",A11V6ZJ2FVQY1D,0700026657,IBRAHIM ALBADI,i liked a lot some time that i haven't play a wonderfull game very simply and funny game verry good game.,Anno 2070,1415836800,0,video_games
6,1.0,0,false,"08 2, 2014",A1KXJ1ELZIU05C,0700026657,Creation27,"I'm an avid gamer, but Anno 2070 is an INSULT to gaming. It is so buggy and half-finished that the first campaign doesn't even work properly and the DRM is INCREDIBLY frustrating to deal with. Once you manage to work your way past the massive amounts of bugs and get through the DRM, HOURS later you finally figure out that the game has no real tutorial, so you stuck just clicking around randomly. Sad, sad, sad, example of a game that could have been great but FTW.",Avoid This Game - Filled with Bugs,1406937600,0,video_games
7,5.0,0,true,"03 3, 2014",A1WK5I4874S3O2,0700026657,WhiteSkull,"I bought this game thinking it would be pretty cool and that i might play it for a week or two and be done. Boy was I wrong! From the moment I finally got the gamed Fired up (the other commentors on this are right, it takes forever and u are forced to create an account) I watched as it booted up I could tell right off the bat that ALOT of thought went into making this game. If you have ever played Sim city, then this game is a must try as you will easily navigate thru it and its multi layers. I have been playing htis now for a month straight, and I am STILL discovering layers of complexity in the game. There are a few things in the game that could used tweaked, but all in all this is a 5 star game.",A very good game balance of skill with depth of choices,1393804800,0,video_games
8,5.0,0,true,"02 21, 2014",AV969NA4CBP10,0700026657,Travis B. Moore,I have played the old anno 1701 AND 1503. this game looks great but is more complex than the previous versions of the game. I found a lot of things lacking such as the sources of power and an inability to store energy with batteries or regenertive fuel cells as buildings in the game need power. Trade is about the same. My main beef with this it requires an internet connection. Other than that it has wonderful artistry and graphics. It is the same as anno 1701 but set in a future world where global warmming as flood the land and resource scarcity has sent human kind to look to the deep ocean for valuable minerals. I recoment the deep ocean expansion or complete if you get this. I found the ai instructor a little corny but other than that the game has some real polish. I wrote my 2 cents worth on suggestions on anno 2070 wiki and you can read 3 pages on that for game ideas I had.,Anno 2070 more like anno 1701,1392940800,0,video_games
9,4.0,0,true,"06 27, 2013",A1EO9BFUHTGWKZ,0700026657,johnnyz3,"I liked it and had fun with it, played for a wh

#Data Pre-Processing and Cleaning and EDA

In [4]:
#Check for nulls
import pyspark.sql.functions as F

nullcheck = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])
display(nullcheck)

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,label,category,reviewDate,review_number,review_number_within_category,Review_Wordcount
0,0,0,0,0,0,241,0,385,0,0,0,0,0,0


In [5]:
#Drop duplicates - deprecated as review edits are allowed

"""print("Before duplication removal: ", df.count())
df = df.dropDuplicates(['reviewerID', 'asin'])
print("After duplication removal: ", df.count())


# Take a sample (useful for code development purposes)
#df = df.sample(False, 0.15, seed=0)
df = df.cache()

print((df.count(), len(df.columns)))"""

In [6]:
import pyspark.sql.functions as f
from pyspark.sql.functions import date_format, row_number, col, from_unixtime, to_date
from pyspark.sql.window import *

#Non Textual Feature Engineering

#1. Time Features

# First, convert Unix timestamp to readable date

df = df.withColumn("reviewDate", to_date(from_unixtime(df.unixReviewTime))) \
                                                .drop("reviewTime","rn")

#Day of week and month of week features
df = df.withColumn("dayOfWeek", date_format(col('reviewDate'), 'E '))
df = df.withColumn("monthOfYear", date_format(col('reviewDate'), 'MMM'))

#2. Review-Based Features
#creation and addition of review_number feature (the nth review of a particular reviewer/customer)
df = df.withColumn("review_number", row_number().over(Window.partitionBy("reviewerID").orderBy("reviewDate"))).sort("reviewerID","review_number")

#taking this a step further, let's look at the review_number within a category
df = df.withColumn("review_number_within_category", row_number().over(Window.partitionBy("reviewerID","category").orderBy("reviewDate"))).sort("reviewerID","category","review_number_within_category")

#Review Number, but at the product level
df = df.withColumn("product_review_number", row_number().over(Window.partitionBy("asin").orderBy("unixReviewTime")))

#total number of words in a review
df = df.withColumn('Review_Wordcount', f.size(f.split(f.col('reviewText'), ' ')))
display(df)

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,reviewDate,dayOfWeek,monthOfYear,review_number,review_number_within_category,product_review_number,Review_Wordcount
17239,5.0,28,false,AGHIV0V0ON7MO,0002151731,David Graham,"Until recently, when asked by his students for an up-to-date, readable, one-volume history of medicine, Roy Porter was at a loss of what to recommend. He therefore decided to bridge the gap, so to speak, and undertake this momentous task himself. In so far as it is possible for someone to adequately accomplish this Herculean task of being both comprehensive and somewhat concise (the material is indeed covered in one volume, though 831 pages long), Roy Porter has succeeded. Porter has an eye for the unusual, spicing up his reporting with examples of odd concoctions and practices used for various maladies down through the ages, such as the use of pulverized crocodile dung, various herbs, and honey as a contraceptive pessary among the ancient Egyptians, or the English resistance against legal revisions (including town sewer reform among other things) attempting to fight cholera in the 19th century: ""We prefer to take our chances with cholera and the rest rath! er than be bullied into health,"" reported THE TIMES. Most refreshingly, he is not timid in rendering pronouncements for both good and ill on the medical profession, bringing a candor needed to assess the impact of medicine down through the ages. He is thorough without being tedious, educational without being pedantic, and has a fine eye for comedy without being flippant. As someone with an interest in history and by vocation a surgeon, I found Roy Porter's book a delightfully instructive volume to read. I look forward to returning to peruse it many times in the years ahead.",Superb Medical History in One Volume,902016000,1,books,1998-08-02,Sun,Aug,40,40,1,266
17238,4.0,9,true,A1TAU5R8BSLKTC,0002151731,some guy on the internet,"There is a solid narrative history of the up to 1800 or so and then he gives extended essays on a variety of topics like surgery, tropical medicine or public medicine. One chapter each on ancient Asian and Islamic medicine. I wish he had fulled out his thesis that as medicne has gotten more successful, medical consumers have gotten more disatisfied with the limits of care.",Encyclopedic Survey of the History of Medicine,943574400,1,books,1999-11-26,Fri,Nov,1,1,2,70
17237,2.0,24,false,A2YQRF5UMH2UTQ,0002151731,Bernard Morey,"I started reading with great relish, looking forward to a well-written history of medicine. I got as far as page 34 before I concluded that this book was never going to live up to its promise and was, in fact, a Marxist polemic. The book does does say who Roy Porter is, but from the tone of the book it seems he is one of those trendy journalists (he surely cannot be a scientist) who detest Western values and particularly Western science. At page 34 readers are informed that ""the rebirth of the underclass in the First World (results) from the free-market policies dominant since the 1980s, (has) assisted the resurgence of disease"". Oh dear. This tosh merely reveals Porter's true colours and shows him to be be short on history but long on leftist cliches. And it goes downhill from there. Shortly after bad health in some of the third world are attributed to that great leftist bugaboo, colonialism. The fact that many dirt poor third world countires got that way through through the machinations of post-colonial home-grown dictators of a particularly vicious and evil kind seems to have entirely escaped Porter's attention. I then skimmed through the rest of the book and similiar throw-away and absurdly wrong opinions abound. Later on we are informed that ""Synthetic food additives and dyes, fertilizers, and pesticides may all be carcinogenic"".",A disappointment,954547200,1,books,2000-04-01,Sat,Apr,1,1,3,253
17236,4.0,44,false,AGG9C66TOLJZB,0002151731,A. Woo

# EDA

In [8]:
# Let's use Spark SQL for some simple exploratory analysis. Firstly, we need to create a temporary view based on the dataframe. This must be done to allow SQL queries to be run against the data
df.createOrReplaceTempView("df_sql")

In [9]:
#Reviews Split by helpfulness
display(df.groupBy("label").count())

label,count
1,626166
0,2861165


In [10]:
# overall rating counts split by helpful and unhelpful
from pyspark.sql.functions import col
display(spark.sql("""select overall, count(CASE WHEN label == 1 THEN 1 END) as helpful, count(CASE WHEN label == 0 THEN 1 END) as unhelpful
           from df_sql
           -- where category = 'home_and_kitchen' and year(reviewDate) = 2001
           group by overall
           order by overall desc """))


overall,helpful,unhelpful
5.0,241,26
4.0,78,16
3.0,33,1
2.0,13,2
1.0,19,3


In [11]:
#total reviews by category
display(spark.sql("""select category as Category, count(distinct reviewerID) as num_reviewers from df_sql group by category"""))

Category,num_reviewers
home_and_kitchen,622941
books,538935
video_games,55197


In [12]:
# category structure
display(df.groupBy("category", "label").count().orderBy("category"))

category,label,count
books,1,33969
books,0,115502
home_and_kitchen,0,255896
home_and_kitchen,1,44327
video_games,1,16001
video_games,0,57081


In [13]:
# Number of reviews over time

review_over_time = spark.sql('''
  SELECT 
    reviewDate AS date, 
    COUNT(*) AS total_review,
    SUM(CASE WHEN label==1 THEN 1 ELSE 0 END) AS total_helpful
  FROM
    df_sql
  GROUP BY
    reviewDate
  ORDER BY
    reviewDate
''')

display(review_over_time)

#helpful review over time without using sql
#helpful_daily = df.filter(F.col("label")==1).groupBy("reviewDate").count().orderBy("reviewDate", ascending=True)

date,total_review,total_helpful
1996-09-18,1,1
1996-11-20,1,1
1996-12-17,1,1
1997-03-08,1,1
1997-04-15,1,1
1997-04-17,1,1
1997-05-03,1,1
1997-05-09,1,1
1997-06-01,1,1
1997-06-05,1,0


In [14]:
# Number of helpful reviews over time standalone

helpful_review_over_time = spark.sql('''
  SELECT 
    reviewDate AS date, 
    SUM(CASE WHEN label==1 THEN 1 ELSE 0 END) AS total_helpful
  FROM
    df_sql
  WHERE
    reviewDate >= '2015-01-01'
  GROUP BY
    reviewDate
  ORDER BY
    reviewDate
''')

display(helpful_review_over_time)

In [15]:
#percent of helpful reviews over total reviews by day of week
import pyspark.sql.functions as F
from pyspark.sql.functions import when,count

helpful_day = df.filter(F.col("label")==1).groupBy("dayOfWeek").count()
helpful_day = helpful_day.select(col("dayOfWeek"), col("count").alias("helpfulCount"))
total_day = df.groupBy("dayOfWeek").count()
total_day = total_day.select(col("dayOfWeek"), col("count").alias("totalCount"))
dow = helpful_day.join(total_day, helpful_day.dayOfWeek == total_day.dayOfWeek, 'inner').drop(helpful_day.dayOfWeek)
dow = dow.withColumn("helpfulPercent", col("helpfulCount") / col("totalCount"))
display(dow.select(col("dayOfWeek"), col("helpfulPercent")))

dayOfWeek,helpfulPercent
Mon,0.17517641242303025
Sun,0.1811235008919774
Tue,0.18201864668274642
Thu,0.17775245831636077
Fri,0.18019497623223615
Wed,0.1797816431277856
Sat,0.18133109347715567


In [16]:
#percent of helpful reviews over total reviews by month
helpful_month = df.filter(F.col("label")==1).groupBy("monthOfYear").count()
helpful_month = helpful_month.select(col("monthOfYear"), col("count").alias("helpfulCount"))
total_month = df.groupBy("monthOfYear").count()
total_month = total_month.select(col("monthOfYear"), col("count").alias("totalCount"))
moy = helpful_month.join(total_month, helpful_month.monthOfYear == total_month.monthOfYear, 'inner').drop(helpful_month.monthOfYear)
moy = moy.withColumn("helpfulPercent", col("helpfulCount") / col("totalCount"))
display(moy.select(col("monthOfYear"), col("helpfulPercent")))

monthOfYear,helpfulPercent
Oct,0.18767656976214345
Sep,0.1768793624116006
Dec,0.1710469687805332
Aug,0.1714692681688532
May,0.191894852135816
Jun,0.19638011990446635
Feb,0.1674895260135146
Nov,0.19912851800538828
Mar,0.1773691833007997
Jan,0.16709187007596155


In [17]:
#Some investigations into reveiewing habits

#does the helpfulness of reviews increase as a customer writes more reviews?

review_performance_over_time = spark.sql("""select review_number
                                                    ,category 
                                                    ,avg(label) as average_helpfulness
                                                    ,count(1) as number_of_occurrences
                                                    from df_sql
                                                    where year(reviewDate) = 2016
                                                    group by review_number, category
                                                    order by category, review_number asc""")
display(review_performance_over_time)

review_number,category,average_helpfulness,number_of_occurrences
1,books,0.07537121842908687,57648
2,books,0.08080935025763479,23871
3,books,0.0876557391534859,11317
4,books,0.09652325765994603,6299
5,books,0.10070031393383241,4141
6,books,0.10334137099552188,2903
7,books,0.11423039690222653,2066
8,books,0.13216453135536074,1483
9,books,0.12478336221837089,1154
10,books,0.13837837837837838,925


In [18]:
display(df)

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,reviewDate,dayOfWeek,monthOfYear,review_number,review_number_within_category,product_review_number,Review_Wordcount
17239,5.0,28,false,AGHIV0V0ON7MO,0002151731,David Graham,"Until recently, when asked by his students for an up-to-date, readable, one-volume history of medicine, Roy Porter was at a loss of what to recommend. He therefore decided to bridge the gap, so to speak, and undertake this momentous task himself. In so far as it is possible for someone to adequately accomplish this Herculean task of being both comprehensive and somewhat concise (the material is indeed covered in one volume, though 831 pages long), Roy Porter has succeeded. Porter has an eye for the unusual, spicing up his reporting with examples of odd concoctions and practices used for various maladies down through the ages, such as the use of pulverized crocodile dung, various herbs, and honey as a contraceptive pessary among the ancient Egyptians, or the English resistance against legal revisions (including town sewer reform among other things) attempting to fight cholera in the 19th century: ""We prefer to take our chances with cholera and the rest rath! er than be bullied into health,"" reported THE TIMES. Most refreshingly, he is not timid in rendering pronouncements for both good and ill on the medical profession, bringing a candor needed to assess the impact of medicine down through the ages. He is thorough without being tedious, educational without being pedantic, and has a fine eye for comedy without being flippant. As someone with an interest in history and by vocation a surgeon, I found Roy Porter's book a delightfully instructive volume to read. I look forward to returning to peruse it many times in the years ahead.",Superb Medical History in One Volume,902016000,1,books,1998-08-02,Sun,Aug,40,40,1,266
17238,4.0,9,true,A1TAU5R8BSLKTC,0002151731,some guy on the internet,"There is a solid narrative history of the up to 1800 or so and then he gives extended essays on a variety of topics like surgery, tropical medicine or public medicine. One chapter each on ancient Asian and Islamic medicine. I wish he had fulled out his thesis that as medicne has gotten more successful, medical consumers have gotten more disatisfied with the limits of care.",Encyclopedic Survey of the History of Medicine,943574400,1,books,1999-11-26,Fri,Nov,1,1,2,70
17237,2.0,24,false,A2YQRF5UMH2UTQ,0002151731,Bernard Morey,"I started reading with great relish, looking forward to a well-written history of medicine. I got as far as page 34 before I concluded that this book was never going to live up to its promise and was, in fact, a Marxist polemic. The book does does say who Roy Porter is, but from the tone of the book it seems he is one of those trendy journalists (he surely cannot be a scientist) who detest Western values and particularly Western science. At page 34 readers are informed that ""the rebirth of the underclass in the First World (results) from the free-market policies dominant since the 1980s, (has) assisted the resurgence of disease"". Oh dear. This tosh merely reveals Porter's true colours and shows him to be be short on history but long on leftist cliches. And it goes downhill from there. Shortly after bad health in some of the third world are attributed to that great leftist bugaboo, colonialism. The fact that many dirt poor third world countires got that way through through the machinations of post-colonial home-grown dictators of a particularly vicious and evil kind seems to have entirely escaped Porter's attention. I then skimmed through the rest of the book and similiar throw-away and absurdly wrong opinions abound. Later on we are informed that ""Synthetic food additives and dyes, fertilizers, and pesticides may all be carcinogenic"".",A disappointment,954547200,1,books,2000-04-01,Sat,Apr,1,1,3,253
17236,4.0,44,false,AGG9C66TOLJZB,0002151731,A. Woo

In [19]:
#same as above, but at the product-review_number level

#does the likelihood of review being helpful decrease if there are already many reviews written for the same product?

review_performance_over_time_product = spark.sql("""select product_review_number
                                                    ,category 
                                                    ,avg(label) as average_helpfulness
                                                    ,count(1) as number_of_occurrences
                                                    from df_sql 
                                                    group by product_review_number, category
                                                    order by category, product_review_number asc""")
display(review_performance_over_time_product)


product_review_number,category,average_helpfulness,number_of_occurrences
1,books,0.8270904879801932,12521
2,books,0.7695113674923652,11788
3,books,0.7187443784853391,11118
4,books,0.6683407561830759,10553
5,books,0.6345981475948611,10041
6,books,0.6011271133375078,9582
7,books,0.566775956284153,9150
8,books,0.539224629418472,8770
9,books,0.5187381404174574,8432
10,books,0.5044313146233382,8124


In [20]:
print((df.count(), len(df.columns)))

(3487331, 19)

In [21]:
#does rating and review length have any relationship?
rating_vs_length = spark.sql("""select overall
                                      ,avg(Review_Wordcount) as average_review_length
                                      ,count(1) as instances
                                from df_sql
                                group by overall
                                order by overall, count(1) asc
                               """)

display(rating_vs_length)

overall,average_review_length,instances
1.0,91.53778328264495,163088
2.0,111.95386718224375,141201
3.0,109.61403677602748,270176
4.0,102.42181273846718,567164
5.0,61.853847181957136,2057545


In [22]:
#does helpfulness and review length have any relationship?
helpful_vs_length = spark.sql("""select overall, label, 
                                avg(Review_Wordcount) as review_Length
                                
                                from df_sql
                                where Review_Wordcount > 5
                                group by overall, label
                                order by overall, label 
                                """)

display(helpful_vs_length)

overall,label,review_Length
1.0,0,60.44957481542848
1.0,1,138.54469028399348
2.0,0,75.73604835129814
2.0,1,194.15682856839197
3.0,0,79.75570726174489
3.0,1,223.3138482632541
4.0,0,80.58004633047126
4.0,1,234.26624795452182
5.0,0,52.28486682106902
5.0,1,176.35759501388324


In [23]:
# The most common product IDs
display(df.groupBy("asin").count().orderBy(col("count").desc()).head(50))

asin,count
0007420412,19655
000711835X,18496
0007548672,15921
0007350899,7890
0007444117,7830
B000YGEVMI,7356
0007378033,6336
B0015TMHSI,5846
006017322X,5161
0007384289,5095


In [24]:
#Multi review buckets
Multi_review_buckets = spark.sql("""
  select category
        ,case when number_of_reviews_in_category = 1 then '1'
              when number_of_reviews_in_category = 2 then '2'
              when number_of_reviews_in_category = 3 then '3'
              when number_of_reviews_in_category = 4 then '4'
              when number_of_reviews_in_category = 5 then '5'
              when number_of_reviews_in_category > 5 and number_of_reviews_in_category <= 10 then '5-10'
              when number_of_reviews_in_category > 10 and number_of_reviews_in_category <= 20 then '11-20'
              when number_of_reviews_in_category > 20 and number_of_reviews_in_category <= 50 then '21-50'
              when number_of_reviews_in_category > 50 then '50+' end as number_of_reviews_in_category
        ,count(reviewerID) as num_reviewers
  from
        (select reviewerID
                ,category
                ,max(review_number_within_category) as number_of_reviews_in_category
          from df_sql
          group by reviewerID, category) as a
  group by category
         , case when number_of_reviews_in_category = 1 then '1'
              when number_of_reviews_in_category = 2 then '2'
              when number_of_reviews_in_category = 3 then '3'
              when number_of_reviews_in_category = 4 then '4'
              when number_of_reviews_in_category = 5 then '5'
              when number_of_reviews_in_category > 5 and number_of_reviews_in_category <= 10 then '5-10'
              when number_of_reviews_in_category > 10 and number_of_reviews_in_category <= 20 then '11-20'
              when number_of_reviews_in_category > 20 and number_of_reviews_in_category <= 50 then '21-50'
              when number_of_reviews_in_category > 50 then '50+' end
  order by category
         , case when number_of_reviews_in_category = 1 then '1'
              when number_of_reviews_in_category = 2 then '2'
              when number_of_reviews_in_category = 3 then '3'
              when number_of_reviews_in_category = 4 then '4'
              when number_of_reviews_in_category = 5 then '5'
              when number_of_reviews_in_category > 5 and number_of_reviews_in_category <= 10 then '5-10'
              when number_of_reviews_in_category > 10 and number_of_reviews_in_category <= 20 then '11-20'
              when number_of_reviews_in_category > 20 and number_of_reviews_in_category <= 50 then '21-50'
              when number_of_reviews_in_category > 50 then '50+' end
         """)

display(Multi_review_buckets)


#compare that to the total number of reviews in each category:
#books: 500k
#home and kitchen: 600k
#video games: 55k

category,number_of_reviews_in_category,num_reviewers
books,1,351645
books,11-20,3782
books,2,106692
books,21-50,1403
books,3,38673
books,4,16225
books,5,8258
books,5-10,11896
books,50+,361
home_and_kitchen,1,165123


In [25]:
#multi reviewers (>= 5 reviews) by category
multi_reviewer_by_category = spark.sql("""
  select z.category
        ,z.num_multi_reviewers
        ,b.num_reviewers
        ,z.num_multi_reviewers/b.num_reviewers as percent_multi_reviewer
  from
  (select category
        ,count(reviewerID) as num_multi_reviewers
  from
        (select reviewerID
                ,category
                ,max(review_number_within_category) as number_of_reviews_in_category
          from df_sql
          group by reviewerID, category) as a        
  where number_of_reviews_in_category >= 5
  group by category) as z
  join 
        (select category
                      ,count(distinct reviewerID) as num_reviewers
                from df_sql
                group by category) as b
  on z.category = b.category
         """)

#books
display(multi_reviewer_by_category)

category,num_multi_reviewers,num_reviewers,percent_multi_reviewer
home_and_kitchen,119954,622941,0.19256077220796192
books,25700,538935,0.04768664124616141
video_games,53370,55197,0.9669003750203815


In [26]:
#customer level analysis

#create the customer level table to investigate customer characteristics like review polarity and total reviews in each category
#review polarity, or the size of the variance of ratings given by a customer or on a product. Measured as the standard deviation of ratings (customer level or product level)

customer_level = spark.sql("""select a.reviewerID
        ,a.total_reviews_across_all_categories
        ,a.overall_polarity_score
        ,b.books_polarity_score
        ,b.home_and_kitchen_polarity_score
        ,b.video_games_polarity_score
        ,b.number_of_book_reviews
        ,b.number_of_home_and_kitchen_reviews
        ,b.number_of_video_game_reviews
from
  (select reviewerID
          ,max(review_number) as total_reviews_across_all_categories
          ,stddev_pop(overall) as overall_polarity_score
  from df_sql 
  group by reviewerID) a
join 
  (select reviewerID
          ,sum(case when category = 'books' then category_polarity_score else 0 end) as books_polarity_score
          ,sum(case when category = 'home_and_kitchen' then category_polarity_score else 0 end) as home_and_kitchen_polarity_score
          ,sum(case when category = 'video_games' then category_polarity_score else 0 end) as video_games_polarity_score
          ,sum(case when category = 'books' then number_of_reviews_in_category else 0 end) as number_of_book_reviews
          ,sum(case when category = 'home_and_kitchen' then number_of_reviews_in_category else 0 end) as number_of_home_and_kitchen_reviews
          ,sum(case when category = 'video_games' then number_of_reviews_in_category else 0 end) as number_of_video_game_reviews
    from
                    (select reviewerID
                                    ,category
                                    ,max(review_number) as total_reviews_across_all_categories
                                    ,max(review_number_within_category) as number_of_reviews_in_category
                                    ,stddev_pop(overall) as category_polarity_score
                    from df_sql 
                    group by reviewerID, category) z
  group by reviewerID) b
on a.reviewerID = b.reviewerID
where total_reviews_across_all_categories >= 5""")

display(customer_level.groupBy('total_reviews_across_all_categories').count().collect())

#display(customer_level)

total_reviews_across_all_categories,count
471,1
623,1
31,246
85,5
65,26
53,39
133,1
78,7
155,4
108,3


In [27]:
#Investigation into the 0 polarity customers

#sanity check after the main query .. seems like these customers legitimately review with the same rating each time!
#display(df.filter(df.reviewerID=='A1EQQIK6QU4JAH'))

#Let's look at the rating distribution of 0 polarity customers
#customer_level=sqlContext.sql("select * from customer_level")
customer_level.createOrReplaceTempView("customer_level")


Consistent_Reviewer_Rating_Breakdown = spark.sql("""select a.*
                                                    from
                                                    (select reviewerID
                                                            , avg(overall) as rating_habit
                                                    from df_sql 
                                                    --where category = 'books'
                                                    group by reviewerID) a
                                                    join customer_level b on a.reviewerID = b.reviewerID
                                                    where b.overall_polarity_score = 0 and total_reviews_across_all_categories >=5 --and b.number_of_book_reviews >=5""")

display(Consistent_Reviewer_Rating_Breakdown)

reviewerID,rating_habit
A101CZ45F288CC,5.0
A10SGV8C0KG3LN,5.0
A11J9Z80LDW5S2,5.0
A11JEFCFYWYV4W,5.0
A11WPBIJOKBHL,5.0
A12CVL3OEBNQNZ,5.0
A12GV0K5PX71W5,5.0
A12WD8M1UUA93G,5.0
A1431KNVPI094S,5.0
A148C90WFQ3W00,5.0


In [28]:
#product level analysis
product_level = spark.sql(""" select * from
                              (select asin as productID
                                    ,category
                                    ,count(1) as reviews
                                    ,avg(overall) as rating
                                    ,stddev_pop(overall) as product_polarity
                                    ,stddev_pop(overall)/avg(overall) as CV_product_polarity
                             from df_sql
                             group by asin,category)
                             where reviews >= 5
                              """)

display(product_level.filter(product_level.product_polarity==0))

productID,category,reviews,rating,product_polarity,CV_product_polarity
B00008KTX6,video_games,5,5.0,0.0,0.0
B009716CFK,video_games,15,5.0,0.0,0.0
B0013TV86M,home_and_kitchen,6,5.0,0.0,0.0
B000XB541Q,home_and_kitchen,18,1.0,0.0,0.0
B001B3UOR4,video_games,16,5.0,0.0,0.0
B003IVUNV4,home_and_kitchen,6,5.0,0.0,0.0
B001EPL14O,home_and_kitchen,5,5.0,0.0,0.0
B001C2RVD4,home_and_kitchen,7,5.0,0.0,0.0
B00005YXRM,video_games,7,5.0,0.0,0.0
0007481152,books,6,5.0,0.0,0.0


In [29]:
#Let's investigate the details of the 5* reviews (code can be/was modified to look at other rating levels, such as 1*). Additional edits made for category/time breakdown
from pyspark.ml.clustering import KMeans
from numpy import array
from math import sqrt
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt

#First, create the 5 star rating review wordcount array dataframe
from pyspark.sql.functions import col
Five_star_review_wordcount_clusters = df[['Review_Wordcount','label']].filter(col("overall")==5) #filter(col("overall")==1


#Assemble the feature vector
vectorAssembler = VectorAssembler(inputCols = ["Review_Wordcount","label"], outputCol = "features")
Five_star_review_wordcount_clusters = vectorAssembler.transform(Five_star_review_wordcount_clusters)

# Build the model (cluster the data)
kmeans = KMeans(maxIter=20,k=6,initMode="k-means||")
model = kmeans.fit(Five_star_review_wordcount_clusters)

# Evaluate clustering by computing Within Set Sum of Squared Errors (a.k.a. WCSS)
wssse = model.computeCost(Five_star_review_wordcount_clusters)
print("Within Set Sum of Squared Errors = " + str(wssse))

######################### #Elbow test and plot for optimal number of review_wordcount clusters within 5* reviews ##########################################
# sse={}

# for k in range(2, 10):
#     kmeans = KMeans(k=k, maxIter=20,initMode="k-means||").fit(Five_star_review_wordcount_clusters)
#     kmeans.transform(Five_star_review_wordcount_clusters).select("prediction")
#     sse[k] = kmeans.computeCost(Five_star_review_wordcount_clusters) 
# fig = plt.figure()
# plt.plot(list(sse.keys()), list(sse.values()))
# plt.xlabel("Number of clusters")
# display(fig)

Within Set Sum of Squared Errors = 2525184996.623439

In [30]:
#show summary stats for the results of the clustering
summary_df = model.transform(Five_star_review_wordcount_clusters).groupBy("prediction").agg({"Review_Wordcount": "avg", "label": "avg","features":"count"}).select(col("avg(Review_Wordcount)").alias("Average Word Count"), col("count(features)").alias("Cluster Size/Number of Reviews"), col("avg(label)").alias("Percentage of Reviews that were Helpful"))
display(summary_df)

Average Word Count,Cluster Size/Number of Reviews,Percentage of Reviews that were Helpful
913.474841992941,12183,0.6947385701387179
91.56347854824865,435722,0.2818494361083443
2041.985873605948,1345,0.7159851301115242
229.65462071282926,130943,0.5288942516973034
473.3858396017946,44801,0.6349188634182272
20.29817383470114,1626085,0.05143642552511093


In [31]:
#Exemplar reviews with Review_wordcount = 22, to give examples of these "low quality" reviews
display(df.filter((df.Review_Wordcount ==33) & (df.overall==1)))

reviewID,overall,vote,verified,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label,category,reviewDate,dayOfWeek,monthOfYear,review_number,review_number_within_category,product_review_number,Review_Wordcount,id
474010,1.0,6,false,A1QYMELGKEAYP1,0027676463,Reggie5,Too bad this book was edited completely out of its classic self. Shame on you! Humbug!! Quit trying to sell us what you think would be good for us. We are not retarded.,Bah Humbug,1384819200,1,books,2013-11-19,Tue,Nov,1,1,744,33,4972
958189,1.0,0,true,A1PT7J4NCE8378,B0012BU6TQ,Sheila A. Quinn,"This arrived yesterday from China. Only one pitiful little pad, not two. It does not fit my Shark. It looks like a large sanitary napkin. Don't bother with this.",Not satisfied,1480896000,0,home_and_kitchen,2016-12-05,Mon,Dec,1,1,449,33,15669
1187009,1.0,0,true,A1PT7J4NCE8378,B0012BU6TQ,Sheila A. Quinn,"This arrived yesterday from China. Only one pitiful little pad, not two. It does not fit my Shark. It looks like a large sanitary napkin. Don't bother with this.",Not satisfied,1480896000,0,home_and_kitchen,2016-12-05,Mon,Dec,2,2,450,33,15670
1479492,1.0,0,true,A2T0JKDW4YL0PQ,B001F51A4Y,LadyBeesai,After few months the cover start to break. It's only good for a year at least. I don't recommend this product waste of money. Get ur self a decent hanger cabinet in ikea.,It's only good for a year at least,1480377600,0,home_and_kitchen,2016-11-29,Tue,Nov,1,1,52,33,17449
8590021259,1.0,0,false,A1BM4GLPZZAPWA,B00DDXILBQ,Christopher,"Theirs nothing to do in this game , but running around finding the next strongest gun or cool clothes. Nothing to this game except a decent story. Highly recommend buy a different game.",Boring...,1472342400,0,video_games,2016-08-28,Sun,Aug,4,4,140,33,21013
362357,1.0,4,true,APPZ8ZSN4S3Y8,B0007X9FN8,Reviews are important,Sorry but this looks cheap and I wound up donating it. It's plastic. Don't bother buying - unless you have kids and they can use it after their tea party lol.,Sorry but this looks cheap and I wound up donating ...,1486512000,1,home_and_kitchen,2017-02-08,Wed,Feb,3,3,45,33,8589939181
563071,1.0,0,true,AFDUY33ODVZX,B000H78J36,Bookworm,"This can opener doesn't grab the can or even puncture the lid. It's great at tearing up the label, though. Listen to the bad reviews and buy a different brand of can opener.",Doesn't work at all.,1480896000,0,home_and_kitchen,2016-12-05,Mon,Dec,3,3,272,33,8589940172
983797,1.0,0,true,AXLUXNI7NERCX,B0013CBGYI,Janice Bryant,I tried these a few times with different items inside. Every single one of the vacuum seals failed after a month or less. I was better off with Rubbermaid plastic containers.,Didn't work,1390435200,0,home_and_kitchen,2014-01-23,Thu,Jan,3,3,91,33,8589943227
1212617,1.0,0,true,AXLUXNI7NERCX,B0013CBGYI,Janice Bryant,I tried these a few times with different items inside. Every single one of the vacuum seals failed after a month or less. I was better off with Rubbermaid plastic containers.,Didn't work,1390435200,0,home_and_kitchen,2014-01-23,Thu,Jan,4,4,92,33,8589943228
1341340,1.0,0,false,A1SUVM5TDOHWEH,B00191X6VY,Alicia McGuire,This is a horrible product. The slipcovers pull and snag on everything. They do not advertise it at all correctly. I had these for a week and they are ruined. Such a disappointment.,CRAP,1368576000,0,home_and_kitchen,2013-05-15,Wed,May,1,1,2,33,8589943465
